In [ ]:
from pathlib import Path
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import pandas as pd

import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)
#from src.d00_utils.dirnames import excl_analysis_dirname
from src.d00_utils import utilities as utils
from src.d00_utils import dirnames as dn

In [ ]:
input_dirpath = input('Please enter the full path for the input directory:\n')
input_dirpath = Path(input_dirpath)

In [ ]:
# caax_live_ch = 0
# caax_fix_ch = 2
# cell_ch = 1
# MBP_ch = 3

# ch_labels = {caax_live_ch: 'CAAX (live)',
#              caax_fix_ch: 'CAAX (fixed)', 
#              cell_ch: 'cell membrane', MBP_ch: 'MBP'}

# ch_save_abbr = {caax_live_ch: 'live_caax', 
#                 caax_fix_ch: 'fix_caax', 
#                 cell_ch: 'memb', 
#                 MBP_ch: 'MBP'}

In [ ]:
caax_live_ch = 1
#caax_fix_ch = 2
cell_ch = 2
#MBP_ch = 3

ch_labels = {caax_live_ch: 'CAAX (live)', 
             cell_ch: 'cell membrane'}

ch_save_abbr = {caax_live_ch: 'live_caax', 
                cell_ch: 'memb'}

In [ ]:
ratio_ch_cmbs = []
ratio_ch_cmbs.append((None, caax_live_ch))

In [ ]:
def save_ratiometric_img(num_ch, denom_ch, img, saveinfo):
    
    (imgname, ch_labels, ch_save_abbr, output_subdirpath) = saveinfo
    info_df = utils.extract_img_info(imgname)
    
    img = np.where(img==0, np.nan, img)
    
    # calculate ratio
    if num_ch is None:
        ratio = 1 / img[:, denom_ch, :, :, :].squeeze()
    else:
        ratio = img[:, num_ch, :, :, :].squeeze() / img[:, denom_ch, :, :, :].squeeze()
    
    # set mean = 1
    ratio = ratio / np.nanmean(ratio)
    bin_ratio = (ratio > 1).astype('bool')
    
    # Set labels
    base_imgname = imgname.split('.')[0]
    UID = info_df['UID'].values[0]
    if num_ch is None:
        num_lab = 1
    else:
        num_lab = {ch_labels[num_ch]}
    
    
    # show image
    fig, axs = plt.subplots(1, 2)
    rg = axs[0].imshow(ratio, norm=mcolors.TwoSlopeNorm(vmin=0,
                                  vcenter=1, vmax=3), cmap='coolwarm', interpolation=None)
    axs[0].set_title(f'normalized {num_lab} / {ch_labels[denom_ch]}')
    axs[0].axis('off')
    plt.colorbar(rg, ax=axs[0], shrink=0.5)
    axs[1].imshow(bin_ratio, cmap='gray', interpolation=None)
    axs[1].axis('off')
    axs[1].set_title(f'normalized {num_lab} / {ch_labels[denom_ch]} > 1')
    UID = info_df['UID'].values[0]
    plt.suptitle(UID)
    plt.tight_layout()
    plt.show()
    
    # Save figures
    output_figspath = output_subdirpath / 'figs'
    output_figspath.mkdir(parents=True, exist_ok=True)
    fig.savefig(output_figspath / f'{base_imgname}.pdf')
    
    # Save ratiometric array
    output_arraypath = output_subdirpath / 'ratiometric_arrays'
    output_arraypath.mkdir(parents=True, exist_ok=True)
    np.save(output_arraypath / f'{base_imgname}.npy', ratio)
    return ratio, bin_ratio

In [ ]:
input_dirpath = Path(input_dirpath)

# Get output directory path
proc_dirpath = utils.get_proc_dirpath(input_dirpath)
output_dirpath = proc_dirpath / dn.analyses_dirname / dn.ratiometric_dirname

imgpaths = [path for path in input_dirpath.glob('*.ome.tif')]
imgpaths.sort()


imgnames = []
cellareas = []
excludedareas = []

for imgpath in imgpaths:
    imgnames.append(imgpath.name)
    
    for cmb in ratio_ch_cmbs:
        
        # input analysis paths
        num_ch = cmb[0]
        denom_ch = cmb[1]

        # Get output subdirectory paths
        if num_ch is None:
            output_subdirpath = output_dirpath / f'over_{ch_save_abbr[denom_ch]}'
        else:
            output_subdirpath = output_dirpath / f'{ch_save_abbr[num_ch]}_over_{ch_save_abbr[denom_ch]}'

        saveinfo = (imgpath.name, ch_labels, ch_save_abbr, output_subdirpath)
        
        img_file = AICSImage(imgpath, reader=OmeTiffReader)
        img = img_file.data
        img = img[0:1, :, :, :, :]
        
        cellarea = np.count_nonzero(img[:, cell_ch, :, :, :])
        cellareas.append(cellarea)
        
        _, bin_ratio = save_ratiometric_img(num_ch, denom_ch, img, saveinfo)
        excludedarea = np.count_nonzero(bin_ratio)
        excludedareas.append(excludedarea)
        
df = pd.DataFrame({'Image name': imgnames, 'Cell area': cellareas, 'Excluded area': excludedareas})
df.to_csv(output_dirpath / 'areas.csv')
df